# Mise en place d'un algorithme de recherche de chemins sur les premiers exemples de vésicules


##### Importation des bibliothéques définis par l'utilisateur

In [1]:
import recherche_chemin
import algoresolution_système

import time
import re #bibliothéque python

def numero(texte):
    if type(texte) is str:
        return recherche_chemin.numero(texte,liste_molecules)
    if type(texte) is list:
        return recherche_chemin.numero2(texte,liste_molecules)

##### Lecture du fichier catalogue pour construire les listes de molécules et réactions

In [14]:
file = "catalog.bc"
nb_réactions_max = 50

elmts = []
inputs = []
enzymes = []
regex = ["(present\()|(, [e\-0-9]+\))|\.", "(MA.*for )|\+|(=>)|\."]
blocs = {"inputs": (regex[0], inputs), "enzymes": (regex[0], enzymes), "elmts": (regex[1], elmts)}

""" Création du tableau "reaction" de l'ensemble des réactions possible données par le fichier file"""
friends = {'test': []}  # Chaque elmt et l'ensemble des elements avec lesquels il reagit
# par contre les enzymes ne sont pas comptés comme éléments, du coup ils ne font pas partie des clés du dict.

reaction = []  # Liste des réactions en version texte
with open(file) as f:
    while f.readline() != "% Inputs\n":
        continue
    bloc = 'inputs'
    while True:
        l = f.readline()
        if not l:
            break

        if l[0] == '%':
            if l.find("Enzymes") > 0:
                bloc = 'enzymes'
                continue
            if l.find('reaction') > 0:
                bloc = 'elmts'
                continue
            continue

        if len(l) < 3:
            continue

        if bloc == 'elmts':
            elmt = l.split('=>')
            reactifs = re.sub("(MA.*for )|\+", " ", elmt[0]).split()
            for r in reactifs:
                try:
                    friends[r].extend([e for e in reactifs if e not in friends[r]])
                except:
                    friends.update({r: [e for e in reactifs if e != r]})
            produits = re.sub("\+|\.", " ", elmt[1]).split()
            reaction.append((reactifs, produits))
            elts = reactifs + produits
        else:
            elts = re.sub(blocs[bloc][0], '', l).split()

        blocs[bloc][1].extend([e for e in elts if (e not in blocs[bloc][1])])

liste_molecules = elmts
for molecule in enzymes :
    if not(molecule in enzymes):
        liste_molecules.append(molecule)
liste_molecules.pop(numero('H_2O_2'))  # Remove 'H_2O_2', 'H2O2' is in list

dictionnaire_molecules={liste_molecules[i] : i for i in range(len(liste_molecules))}

REACTIONS = []  # Liste des réactions en version numéros
for a in reaction:
    reac = ([], [])
    for m in a[0]:
        if m == 'H_20_2' or m == 'H202' or m == 'H_2O_2':
            reac[0].append(recherche_chemin.numero('H2O2',liste_molecules))
        else:
            reac[0].append(recherche_chemin.numero(m,liste_molecules))
    for m in a[1]:
        if m == 'H_20_2' or m == 'H202' or m == 'H_2O_2':
            reac[1].append(recherche_chemin.numero('H2O2',liste_molecules))
        else:
            reac[1].append(recherche_chemin.numero(m,liste_molecules))
    REACTIONS.append(reac)


REACPARMOL = [] # REACPARMOL = ?
for k in range(0, len(liste_molecules)):
    REACPARMOL.append([])
for k in range(0, len(REACTIONS)):
    u = REACTIONS[k][0][0]
    REACPARMOL[u].append(k)
recherche_chemin.REACPARMOL=REACPARMOL

In [3]:
def time_exec(nom_fonction):
    start_time = time.time()
    fonction = globals()[nom_fonction]
    fonction()
    print("Temps de calcul : ",time.time()-start_time)

## Résolution d'exemples

In [27]:
##glucose et acetone donnent gluconolacrone
ENZ = ['AO', 'ADH', 'G_1DH', 'NAD', 'resazurin', 'HRP', 'H2O2']  ##rajouter NAD pour fausser le résultat
#ENZ=liste_molecules[:dictionnaire_molecules.get('resorufin')]+liste_molecules[(dictionnaire_molecules.get('resorufin')+1):]

MECAS = recherche_chemin.résolution_équation(ENZ,"acetoneext + glucoseext => resorufin",nb_réactions_max,reaction,liste_molecules,REACTIONS,REACPARMOL)


PRESENCE :  [True, [(10, 3, 'e'), (10, 12, 'ab')], ['e', 'ab'], [(10, 'e'), (10, 'ab')]]

Les mécanismes après sélection sont les suivants
[([[6], [8], [10]], 'e', [12, 64, 14]), ([[24, 0, 1], [26], [28], [18], [20], [22], [23], [14], [16], [6], [8], [10]], 'ab', [12, 21, 25, 30, 29, 14])]
 
mécanisme 1:
HRP + H_2O_2 -> CCia5
CCia5 + resazurin -> CCib5
CCib5 -> HRP + resorufin
 
mécanisme 2:
G_1DH + NAD -> CCia1 --- glucoseext -> glucose --- acetoneext -> acetone
CCia1 + glucose -> CCib1
CCib1 -> CCfa1 + NADH
ADH + NADH -> CCia2
CCia2 + acetone -> CCib2
CCib2 -> CCfa2 + NAD
CCfa2 -> ADH + isopropanol
AO + isopropanol -> CCf3
CCf3 -> CCio3 + H_2O_2
HRP + H_2O_2 -> CCia5
CCia5 + resazurin -> CCib5
CCib5 -> HRP + resorufin
 
********************************************
['HRP + H_2O_2 -> CCia5']
['CCia5 + resazurin -> CCib5']
['CCib5 -> HRP + resorufin']




In [31]:
solution

[[[30, 25, 18]], [[[18, 1, 0], [20], [22], [24], [26], [28], [29]]]]

In [35]:
solution[0]

[[30, 25, 18]]

In [28]:
## Test lescture équation
ENZ=['ABTS','ADH', 'NADH', 'resazurin', 'HRP', 'AO', 'HRP2', 'POD', 'NR', 'G_1DH', 'O2', 'DAF','NAD'] ##rajouter NAD pour fausser le résultat
RE=['acetoneext','glucoseext']
re=numero(RE)
enz=numero(ENZ)

solution=algoresolution_système.ressystem([re],[numero('gluconolacrone')],['ab'],20,enz,reaction,liste_molecules,REACTIONS,REACPARMOL)

mt = recherche_chemin.mecatexte(solution[0],reaction)
for d in mt[0]:
    print(d)
print("")
print("")

Les mécanismes après sélection sont les suivants
[([[24, 0], [26], [28], [29]], 'b', [30, 29]), ([[24, 0], [26], [28], [29]], 'b', [30, 29]), ([[18, 1, 0], [20], [22], [24], [26], [28], [29]], 'ab', [30, 25, 18]), ([[18, 1, 0], [20], [22], [24], [26], [28], [29]], 'ab', [30, 25, 18])]
 
********************************************
NO -> volatNO
CCia1 -> G_1DH + NAD
ADH + NADH -> CCia2




In [36]:
##NO et glucose donnent gluconolacrone avec NO3 en réactif annexe
ENZ = ['ABTS', 'ADH', 'resazurin', 'HRP', 'AO', 'HRP2', 'POD', 'NR', 'G_1DH', 'O2', 'DAF', 'NAD']
recherche_chemin.résolution_équation(ENZ,"NO2 + glucoseext => DAFF",nb_réactions_max,reaction,liste_molecules,REACTIONS,REACPARMOL)

PRESENCE :  [True, [(39, 11, 'ab')], ['ab'], [(39, 'ab')]]

Les mécanismes après sélection sont les suivants
[([[40, 24, 0, 40, 24, 0], [26, 26], [28, 28], [44, 44], [48, 48], [31], [33], [34], [36], [37], [39]], 'ab', [42, 45, 30, 29, 37, 45, 30, 29])]
 
mécanisme 1:
NR + NO2 -> Cia3 --- G_1DH + NAD -> CCia1 --- glucoseext -> glucose --- NR + NO2 -> Cia3 --- G_1DH + NAD -> CCia1 --- glucoseext -> glucose
CCia1 + glucose -> CCib1 --- CCia1 + glucose -> CCib1
CCib1 -> CCfa1 + NADH --- CCib1 -> CCfa1 + NADH
Cia3 + NADH -> Cfa3 --- Cia3 + NADH -> Cfa3
Cfa3 -> Cio3 + NO --- Cfa3 -> Cio3 + NO
O2 + NO -> Cf6
Cf6 -> O2 + NO2b
NO + NO2b -> Cf5
Cf5 -> N2O3
DAF + N2O3 -> Cf4
Cf4 -> DAFF
 
********************************************
['NR + NO2 -> Cia3', 'G_1DH + NAD -> CCia1', 'glucoseext -> glucose', 'NR + NO2 -> Cia3', 'G_1DH + NAD -> CCia1', 'glucoseext -> glucose']
['CCia1 + glucose -> CCib1', 'CCia1 + glucose -> CCib1']
['CCib1 -> CCfa1 + NADH', 'CCib1 -> CCfa1 + NADH']
['Cia3 + NADH -> Cfa

In [9]:
## Ancien OU logique à changer sur le fonctionnement pluscourtchemin

ENZ=numero(['ABTS', 'ADH', 'NAD', 'resazurin', 'HRP', 'AO', 'HRP2', 'POD', 'NR', 'G_1DH', 'O2', 'DAF', 'LO'])
#recherche_chemin.ENZ = ['ABTS', 'ADH', 'NAD', 'resazurin', 'HRP', 'AO', 'HRP2', 'POD', 'NR', 'G_1DH', 'O2', 'DAF', 'LO']
RE = numero(['Lactateext', 'EtOHext'])

recherche_chemin.résolution_équation(ENZ,"Lactateext + EtOHext => ABTSOX",nb_réactions_max,reaction,liste_molecules,REACTIONS,REACPARMOL)

MECAS = recherche_chemin.pluscourtchemin(ENZ, RE, numero('ABTSOX'), nb_réactions_max, True,reaction,liste_molecules,REACTIONS,REACPARMOL)  # Pourquoi tag a ? OU logique ?
mt = recherche_chemin.mecatexte(MECAS[0][0],reaction)
for d in mt:
    print(d)

TypeError: list indices must be integers or slices, not NoneType

### Test du non

In [ ]:
##glucose et Non(acetone) donnent gluconolacrone
ENZ = ['AO', 'ADH', 'G_1DH', 'NAD', 'resazurin', 'HRP', 'H2O2']  ##rajouter NAD pour fausser le résultat
recherche_chemin.résolution_équation(ENZ,"!acetoneext + glucoseext => NADH",nb_réactions_max,reaction,liste_molecules,REACTIONS,REACPARMOL)

In [ ]:
##glucose et acetone donnent gluconolacrone
ENZ = ['AO', 'ADH', 'G_1DH', 'NAD', 'resazurin', 'HRP', 'H2O2']  ##rajouter NAD pour fausser le résultat
recherche_chemin.résolution_équation(ENZ,"acetoneext + glucoseext => resorufin",nb_réactions_max,reaction,liste_molecules,REACTIONS,REACPARMOL)